In [3]:
import gzip
import json
import random
from collections import defaultdict
import matplotlib.pyplot as plt

def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf8"):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')
path = '/Users/louis/Downloads/redditSubmissions.csv.gz'

In [4]:
dataset = []
for d in readCSV(path):
    print(d)
    break

['0', '1333172439', '2012-03-31T12:40:39.590113-07:00', "And here's a downvote.", '63470', 'rmqjs', '32657', 'funny', '30813', '1333197639', '1844', '622', 'Animates_Everything']
